In [16]:
%load_ext autoreload
%autoreload 2

#import all required modules
from detection.core import DLModel
from modules.xstream import Stream
from modules.xpolygon import XPolygon
from modules.xutils import Config, add_polygon
from modules.xenum import StreamType, ModelType, PolygonType

In [ ]:
#global vars
config = Config()
config.cam_windows_size = (1280, 720)
config.show_windows_size = (1280, 720)

In [ ]:
#define stream object
stream = Stream(stream_type = StreamType.file, config = config)
stream.set_file_location('./assets/videos/vid001.mp4')
cap = stream.get_cap()

[msg]: target video file was set.
[msg]: loading cv2 capture object.
[msg]: capture object loading completed.


In [ ]:
polygon = XPolygon(cap=cap, 
                   polygon_type=PolygonType.line, 
                   show_windows_size=config.show_windows_size)
polygons_list = polygon.draw()

[msg]: extract first frame for drawing polygon shape.
[msg]: drawing risk area as polygon shape.


In [ ]:
# polygons_list=[[(928, 218), (737, 190), (435, 155), (195, 143), (30, 315), (373, 350), (740, 396), (903, 427), (929, 219)]]

In [ ]:
dlmodel = DLModel(model_type=ModelType.yolov8l, 
                  stream=stream, 
                  config=config)
dlmodel.set_risk_area(polygons_list)
dlmodel.detect(extract=True, save_file=False)

[msg]: polygon: POLYGON ((182 141, 386 148, 766 192, 938 215, 888 424, 435 386, 142 337, 24 309, 180 140, 182 141))
[msg]: loading yolo model from .pt file.
[msg]: yolo model loaded successfully.


In [38]:
#import all required modules
import time
import RPi.GPIO as GPIO
from threading import Thread

#base class for managing GPIO process
class XGPIO(object):
    def __init__(self, pins, sra=True): #sra=start signal thread right away
        self.gpio = GPIO
        self.pins = pins
        self.running = True
        self.send_signal_thread = None
        self.current_state = GPIO.HIGH
        self.setup()
    
    def setup(self):
        self.gpio.setmode(GPIO.BCM)
        self.gpio.setup(self.pins, GPIO.OUT, initial=GPIO.HIGH)

    def release(self):
        self.running = True
        self.gpio.cleanup()
    
    def send(self, pins):
        assert len(pins) != 0, 'incorrect pins number list.'
        #repeatedly send signal to the tower
        for i in range(30):
            if self.running:
                self.gpio.output(pins, GPIO.LOW)
                time.sleep(0.1)
            else:
                break

In [39]:
xgpio = XGPIO([5, 12, 18])

In [40]:
xgpio.send([18])